# Analysis of Seattle Collision Data
#### Capstone Project For IBM's Data Science Professional Certificate On Coursera
#### Prepared by Miranda Childs on August 24th, 2020

## Table of contents
* [Introduction and Business Understanding](#introduction)
* [Data Understanding and Preparation](#data)
* [Methodology](#method)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## Introduction and Business Understanding<a name="introduction"></a>

In Seattle, from 2004 to present there have been 194,673 collisions reported by the Seattle Police Department to the Seattle Department of Transportation (SDOT). 58,188 of those collisions resulted in an injury.We aim to reduce the number of accidents, especially those with injuries, in order to increase the wellbeing and longevity of our community. We will prepare a presentation for SDOT and the Vision Zero Network, "a collaborative campaign helping communities reach their goals of Vision Zero -- eliminating all traffic fatalities and severe injuries -- while increasing safe, healthy, equitable mobility for all." (<a href="https://visionzeronetwork.org">Vision Zero Network</a>)


## Data Understanding and Preparation<a name="data"></a>

We will be using the shared data set on collisions from 2004 to present, provided by the Traffic Records Group in conjunction with the Seattle Police Department and Seattle Department of Transportation. (Here are links to <a href="https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv">the dataset</a> and <a href="https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf">corresponding metadata</a>.)


In [2]:
#Import pandas and numpy
import pandas as pd
import numpy as np

In [3]:
#Import the dataset
df = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv");
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
#We can obtain a list of the columns in the dataset
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [5]:
#And review information on the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

In [6]:
#As the collision's severity will serve as our independent variable, it is important to understand this attribute.
#It appears there are three different columns, relating to severity so let's review them.
df_severity= df[['SEVERITYCODE','SEVERITYCODE.1', 'SEVERITYDESC' ]]
df_severity.head()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC
0,2,2,Injury Collision
1,1,1,Property Damage Only Collision
2,1,1,Property Damage Only Collision
3,1,1,Property Damage Only Collision
4,2,2,Injury Collision


In [7]:
#'SEVERITYCODE.1' is a duplicate of the data in 'SEVERITYCODE', and 'SEVERITYDESC' 
#is merely an explanation that 2= injury and 1= property damage only, so we can drop both of those columns

df= df.drop(['SEVERITYDESC', 'SEVERITYCODE.1'], axis=1)
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [8]:
#Let's take another look at the remaining columns
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
       'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE',
       'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL',
       'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM',
       'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR'],
      dtype='object')

In [9]:
#And let's also take a look at the null values in each column
df.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

### Unfortunately, the values for the attributes Speeding, Pedestrian Way Not Granted, and Inattention contain too many null values to be included. 

### Luckily, Under The Influence, Weather, Road Conditions, and Light Conditions all have substantial entries to work with. 

In [10]:
#Let's create a new dataframe with the columns with these features.
df_clean= df[['SEVERITYCODE', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND']]
df_clean.head()

,SEVERITYCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND
0,2,N,Overcast,Wet,Daylight
1,1,0,Raining,Wet,Dark - Street Lights On
2,1,0,Overcast,Dry,Daylight
3,1,N,Clear,Dry,Daylight
4,2,0,Raining,Wet,Daylight


In [11]:
#Let's drop the null values
df_clean.dropna(inplace= True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
#And confirm that the drop was successful
df_clean.isnull().sum()

SEVERITYCODE    0
UNDERINFL       0
WEATHER         0
ROADCOND        0
LIGHTCOND       0
dtype: int64

### Let's review the unique elements in our features:

In [73]:
df_clean['UNDERINFL'].unique()

array(['N', '0', '1', 'Y'], dtype=object)

In [74]:
df_clean['WEATHER'].unique()

array(['Overcast', 'Raining', 'Clear', 'Unknown', 'Other', 'Snowing',
       'Fog/Smog/Smoke', 'Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [75]:
df_clean['ROADCOND'].unique()

array(['Wet', 'Dry', 'Unknown', 'Snow/Slush', 'Ice', 'Other',
       'Sand/Mud/Dirt', 'Standing Water', 'Oil'], dtype=object)

In [76]:
df_clean['LIGHTCOND'].unique()

array(['Daylight', 'Dark - Street Lights On', 'Dark - No Street Lights',
       'Unknown', 'Dusk', 'Dawn', 'Dark - Street Lights Off', 'Other',
       'Dark - Unknown Lighting'], dtype=object)

### And drop rows in which the conditions were 'Unknown' or 'Other'

In [80]:
df_clean.drop(df_clean.loc[df_clean['WEATHER']=='Other'].index, inplace=True)
df_clean.drop(df_clean.loc[df_clean['WEATHER']=='Unknown'].index, inplace=True)
df_clean['WEATHER'].unique()

array(['Overcast', 'Raining', 'Clear', 'Snowing', 'Fog/Smog/Smoke',
       'Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [81]:
df_clean.drop(df_clean.loc[df_clean['ROADCOND']=='Other'].index, inplace=True)
df_clean.drop(df_clean.loc[df_clean['ROADCOND']=='Unknown'].index, inplace=True)
df_clean['ROADCOND'].unique()

array(['Wet', 'Dry', 'Snow/Slush', 'Ice', 'Sand/Mud/Dirt',
       'Standing Water', 'Oil'], dtype=object)

In [82]:
df_clean.drop(df_clean.loc[df_clean['LIGHTCOND']=='Other'].index, inplace=True)
df_clean['LIGHTCOND'].unique()

array(['Daylight', 'Dark - Street Lights On', 'Dark - No Street Lights',
       'Unknown', 'Dusk', 'Dawn', 'Dark - Street Lights Off',
       'Dark - Unknown Lighting'], dtype=object)

### And convert 'Y's to '1's, and 'N's to '0's in the UNDERINFL column 

In [83]:
df_clean['UNDERINFL']= df_clean['UNDERINFL'].replace(['Y'],'1') 
df_clean['UNDERINFL']= df_clean['UNDERINFL'].replace(['N'],'0')
df_clean.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,SEVERITYCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND
0,2,0,Overcast,Wet,Daylight
1,1,0,Raining,Wet,Dark - Street Lights On
2,1,0,Overcast,Dry,Daylight
3,1,0,Clear,Dry,Daylight
4,2,0,Raining,Wet,Daylight


In [84]:
#We confirm that all elements in the UNDERINFL column are now 0s and 1s
df_clean['UNDERINFL'].unique()

array(['0', '1'], dtype=object)

In [85]:
#As you can see in the line above, the datatype for UNDERINFL is still object, so we perform astype() and change those values to int64
df_clean['UNDERINFL'] = df_clean['UNDERINFL'].astype(int)
df_clean['UNDERINFL'].dtype

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


dtype('int64')

### Using the unique function again to confirm that the empty values have been successfully removed:

In [93]:
df_clean['WEATHER'].unique()

array(['Overcast', 'Raining', 'Clear', 'Snowing', 'Fog/Smog/Smoke',
       'Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [97]:
df_clean['LIGHTCOND'].unique()

array(['Daylight', 'Dark - Street Lights On', 'Dark - No Street Lights',
       'Unknown', 'Dusk', 'Dawn', 'Dark - Street Lights Off',
       'Dark - Unknown Lighting'], dtype=object)

In [98]:
df_clean['ROADCOND'].unique()

array(['Wet', 'Dry', 'Snow/Slush', 'Ice', 'Sand/Mud/Dirt',
       'Standing Water', 'Oil'], dtype=object)

### NOW WHAT??? Do we need to perform Label Encoding on the Weather, Road Conditions, and Light Conditions columns so that we may use machine learning? Or can we leave them as categorical variables and use a decision tree or something else???

In [101]:
#So, I looked up the lab from class on Decision Trees because I read somewhere that they were better for categorical variables.
#But then I saw that I still needed to convert the data into numerical values 
#(at this point I was wondering why there was any advantage over other algorithms, 
#but i was already looking for ways to encode my data so I decided to try) 

#The thing I was most confused about here is what values should be held within the X[0:?] . . . 
#I tried different numbers but I couldn't tell what that value was doing, or really this whole piece of code
#(And furthermore-- why does it say "ValueError: y contains previously unseen labels: 'Daylight'" below when I didn't even specify Y yet?")

X = df_clean[['ROADCOND', 'WEATHER', 'LIGHTCOND']].values
X[0:4]

array([['Wet', 'Overcast', 'Daylight'],
       ['Wet', 'Raining', 'Dark - Street Lights On'],
       ['Dry', 'Overcast', 'Daylight'],
       ['Dry', 'Clear', 'Daylight']], dtype=object)

In [102]:
#Once and a great while, I read my errors and I understand all or parts of them. 
#This is not one of these times. 

from sklearn import preprocessing
le_weather = preprocessing.LabelEncoder()
le_weather.fit(['Overcast', 'Raining', 'Clear', 'Snowing', 'Fog/Smog/Smoke','Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt','Severe Crosswind', 'Partly Cloudy'])
X[:,2] = le_weather.transform(X[:,2]) 


le_ROADCOND = preprocessing.LabelEncoder()
le_ROADCOND.fit(['Wet', 'Dry', 'Snow/Slush', 'Ice', 'Sand/Mud/Dirt','Standing Water', 'Oil'])
X[:,3] = le_ROADCOND.transform(X[:,3])


le_LIGHTCOND = preprocessing.LabelEncoder()
le_LIGHTCOND.fit(['Daylight', 'Dark - Street Lights On', 'Dark - No Street Lights','Unknown', 'Dusk', 'Dawn', 'Dark - Street Lights Off','Dark - Unknown Lighting'])
X[:,4] = le_LIGHTCOND.transform(X[:,4]) 

X[0:4]

ValueError: y contains previously unseen labels: 'Daylight'

### Since the code above didn't work, I didn't continue to the next step

In [23]:
#I had actually tried this much earlier but the example I was following didn't have as much information.
#And I did actually create that numerical array;
#But then I saw those numbers and I thought okay but NOW WHAT??? 
#What use is finding out I have a bunch of 3s and 5s if I can't remember which is rain and which is overcast?
#Do we keep the original strings somewhere else to refer to? Or do we encode them back at some point?

from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 
  
df_clean['WEATHER']= label_encoder.fit_transform(df_clean['WEATHER']) 
  
df_clean['WEATHER'].unique() 

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([3, 5, 1, 8, 2, 7, 0, 6, 4])

In [24]:
df_clean

,SEVERITYCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND
0,2,0,3,Wet,Daylight
1,1,0,5,Wet,Dark - Street Lights On
2,1,0,3,Dry,Daylight
3,1,0,1,Dry,Daylight
4,2,0,5,Wet,Daylight
5,1,0,1,Dry,Daylight
6,1,0,5,Wet,Daylight
7,2,0,1,Dry,Daylight
8,1,0,1,Dry,Daylight
9,2,0,1,Dry,Daylight


In [26]:
#I made this test dataframe because I was getting so frustrated, I thought at the very least I could see if I could visualize it somehow
df_test= df_clean[['SEVERITYCODE', 'UNDERINFL', 'WEATHER']]
df_test.head()

,SEVERITYCODE,UNDERINFL,WEATHER
0,2,0,3
1,1,0,5
2,1,0,3
3,1,0,1
4,2,0,5


In [27]:
#I thought this code was supposed to help me choose features and I'm not sure why it's not working. 
#Maybe I need to put the X variable on the side or something instead of being another column??? 
#I'm sure there's something there, but I can't remember.
#On the other hand, Future Warning is a fantastic band name

X = df_test.loc[:,'SEVERITYCODE']  #independent columns
y = df_test.loc[:,'UNDERINFL':'WEATHER']    #target column i.e price range
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: Expected 2D array, got 1D array instead:
array=[2. 1. 1. ... 2. 2. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## I have to figure out how to encode, while saving the values somewhere because the way i am doing it now, none of this means anything! (One more time for the people in the balcony?)

In [243]:
#Create a new column called ROADCOND1 that will hold the numerical values for ROADCOND and perform encoding.

label_encoder = preprocessing.LabelEncoder() 
  
df_clean['ROADCOND1']= df_clean['ROADCOND']= label_encoder.fit_transform(df_clean['ROADCOND']) 
  
df_clean['ROADCOND'].unique() 

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([6, 0, 4, 1, 3, 5, 2])

In [244]:
df_clean

,SEVERITYCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ROADCOND1
0,2,0,3,6,Daylight,6
1,1,0,5,6,Dark - Street Lights On,6
2,1,0,3,0,Daylight,0
3,1,0,1,0,Daylight,0
4,2,0,5,6,Daylight,6
5,1,0,1,0,Daylight,0
6,1,0,5,6,Daylight,6
7,2,0,1,0,Daylight,0
8,1,0,1,0,Daylight,0
9,2,0,1,0,Daylight,0


## If all the encoding stuff had made sense (and it was until I worried about deriving meaning from the numbers later), I would have then moved on to balancing the data. I was thinking of undersampling, but reading about that described it as the 4th or 5th best option. P.S. I don't actually remember learning to balance data either, but since I figured out how to drop columns, it seems far less obscure than the other stuff.

In [52]:
#A closer look at the Severity Code column reveals that the data is imbalanced
df_clean['SEVERITYCODE'].value_counts()

1    116342
2     55920
Name: SEVERITYCODE, dtype: int64

In [14]:
#As there are far fewer Severity Code type 2 entries, we need to balance the data in order to work with it.
#We will do this by undersampling.

## Methodology<a name="method"></a>

## Results<a name="results"></a>

## Discussion<a name="discussion"></a>

## Conclusion<a name="conclusion"></a>

# Wow, if you actually made it this far, you deserve an award. This isn't that, but <a href="https://www.youtube.com/watch?v=Y2vS8dPMR2U">here is something</a>